# Overview

This notebook contain the relevance score experiment for the study

- Dataset that is used is the 9 years stock data that has been formed
- Model that is used in the study leverage an ensemble learning architecture, using LSTM, GRU, and Transformer as base learners, and Multi Layer Perceptron (MLP) as meta learner.

# Import and Define Constant

In [ ]:
import re
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Dropout, GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import L2
from matplotlib.ticker import MultipleLocator
from tqdm.auto import tqdm

# DEFINE CONSTANT

# constant for train size, the rest of the data is used for test
TRAIN_SIZE = 0.85

N_COMPONENT_PCA = 0.8
TARGET_COLUMN_NAME = "Closing Price"

# VARIABLE IN THIS EXPERIMENT
RELEVANCE_COLUMN_USED = "naive"
# RELEVANCE_COLUMN_USED = "relevance_score_linear"
# RELEVANCE_COLUMN_USED = "relevance_score_exponential"
# RELEVANCE_COLUMN_USED = "relevance_score_logarithmic"

# variasi sentiment
EMBEDDING_COLUMN_NAME = "text_embedding_multilingual_mpnet"
USE_PCA = True

LEARNING_RATE_LSTM_GRU = 0.001
N_ITERATION = 20
CONTEXT_WINDOW = 5
NUM_EPOCHS = 100
LOSS = "mse"
METRICS = "mape"

# constant for transformer model
NUM_TRANSFORMER_HEADS = 4
NUM_TRANSFORMER_LAYERS = 2
NUM_TRANSFORMER_HIDDEN_DIM = 32
LEARNING_RATE_TRANSFORMER = 0.0001

fundamental_features  = [
  "Financing Cash Flow",
  "P/B Ratio",
  "P/S Ratio",
  "Capital Adequacy Ratio",
  "Debt to Assets Ratio",
  "Debt to Equity Ratio",
  "Investing Cash Flow",
  "Operating Cash Flow",
  "Return on Assets",
  "Operating Profit",
  "Loan to Deposit Ratio"
]

historic_price_features = [
    'Opening Price',
    'Highest Price',
    'Lowest Price',
    'Volume',
    'Change'
]

relevance_score_features = [
    'relevance_score_linear',
    'relevance_score_logarithmic',
    'relevance_score_exponential'
]

print(f"Number of fundamental features: {len(fundamental_features)}")
print(f"Number of historic price features: {len(historic_price_features)}")
print(f"Number of relevance score variance: {len(relevance_score_features)}")

# Define Function

## Parse Embedding From Pandas Dataframe

In [ ]:
def parse_embedding_from_df(df, embedding_column_name, n_comp, target_column_name="Closing Price"):

  feature_columns = fundamental_features  + historic_price_features + relevance_score_features

  column = [target_column_name, embedding_column_name] + feature_columns
  df_processed = df[column].copy()

  def parse_embedding_string(embedding_str):

      if not isinstance(embedding_str, str):
          return None # Handle non-string inputs (like NaN)

      # Remove brackets and split by whitespace
      embedding_str = embedding_str.strip().strip('[]')
      # Use regex to find all floating point numbers, including those in scientific notation
      numbers = re.findall(r"[-+]?\d*\.?\d+[eE][-+]?\d+|[-+]?\d*\.\d+|\d+", embedding_str)

      try:
          # Convert the extracted numbers to floats
          return [float(num) for num in numbers]
      except ValueError:
          return None # Return None if conversion to float fails for any number

  # Apply the parsing function to the embedding column and handle potential None values
  df_processed['embedding'] = df_processed[embedding_column_name].apply(parse_embedding_string)

  # Handle rows where parsing failed (e.g., by filling with zeros)
  # Determine the embedding dimension from the first successfully parsed embedding
  embedding_dim = None
  for embedding_list in df_processed['embedding']:
      if embedding_list is not None:
          embedding_dim = len(embedding_list)
          break

  if embedding_dim is None:
      # Handle case where all embeddings are None or invalid
      # Using a default BERT base dimension as a fallback.
      embedding_dim = 768 # Default BERT base dimension
      print(f"Warning: Could not determine embedding dimension from data. Using a default embedding dimension of {embedding_dim}.")

  df_processed['embedding'] = df_processed['embedding'].apply(lambda x: np.array(x, dtype=float) if x is not None else np.zeros(embedding_dim))

  def pca_reduce(embedding_list, n_comp=n_comp):
    old_length = len(embedding_list)
    pca = PCA(n_components=n_comp)
    embedding_reduced = pca.fit_transform(np.array(embedding_list.tolist())) # Convert list of arrays to numpy array
    print(f"Old length of embedding: {old_length}")
    print(f"Number of components to explain {n_comp*100}% variance: {pca.n_components_}")
    return embedding_reduced

  df_processed['embedding_pca'] = pca_reduce(df_processed['embedding']).tolist()

  # Convert feature columns to numeric, coercing errors, and fill NaNs
  for col in feature_columns:
      df_processed[col] = pd.to_numeric(df_processed[col], errors='coerce')

  return df_processed

## Create the Train, Test, and Val Data

In [ ]:
def create_data(df_processed, use_pca, embedding_column_name, relevance_column, target_column_price="Closing Price", context_window=5):

    def create_sequences(data, target_index, context_window):
      X, y = [], []
      for i in range(len(data) - context_window):
        X.append(data[i:i+context_window, :])
        y.append(data[i+context_window, target_index])
      return np.array(X), np.array(y)

    # Separate target variable and features
    target_data = df_processed[target_column_price].values

    # exclude irrelevant column from the feature_list
    relevance_columns = ['relevance_score_linear', 'relevance_score_exponential', 'relevance_score_logarithmic']

    if RELEVANCE_COLUMN_USED != 'naive':
      relevance_columns.remove(RELEVANCE_COLUMN_USED)

    feature_columns = [col for col in df_processed.columns if col not in ([target_column_price, embedding_column_name, 'Date', 'embedding', 'embedding_pca'] + relevance_columns)]

    feature_data = df_processed[feature_columns].values

    if use_pca:
      embedding_data = np.array(df_processed['embedding_pca'].tolist())
    else:
      embedding_data = np.array(df_processed['embedding'].tolist())

    print("Feature used in the dataset other than sentiment: ")
    for index, col in enumerate(feature_columns):
      print(f"{index+1}. {col}")

    scaler = MinMaxScaler()
    scaled_target_data = scaler.fit_transform(target_data.reshape(-1, 1))

    feature_scaler = MinMaxScaler()
    scaled_feature_data = feature_scaler.fit_transform(feature_data)

    combined_data = np.concatenate((scaled_target_data, embedding_data, scaled_feature_data), axis=1)

    # Calculate split indices
    n_total = len(combined_data) - CONTEXT_WINDOW
    train_split_index = int(n_total * TRAIN_SIZE)

    train_data = combined_data[:train_split_index + CONTEXT_WINDOW]
    test_data = combined_data[train_split_index:]

    train_generator = TimeseriesGenerator(train_data, train_data[:, 0], # Target is the first column (scaled 'close')
                                        length=CONTEXT_WINDOW, batch_size=24)

    test_generator = TimeseriesGenerator(test_data, test_data[:, 0], # Target is the first column (scaled 'close')
                                      length=CONTEXT_WINDOW, batch_size=24)

    X, y = create_sequences(combined_data, target_index=0, context_window=context_window)

    X_tensor = torch.tensor(X, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.float32)

    X_train, X_test = X_tensor[:train_split_index], X_tensor[train_split_index:]
    y_train, y_test = y_tensor[:train_split_index], y_tensor[train_split_index:]

    train_dataset = TensorDataset(X_train, y_train)
    test_dataset = TensorDataset(X_test, y_test)

    train_loader = DataLoader(train_dataset, batch_size=32)
    test_loader = DataLoader(test_dataset, batch_size=32)

    return train_generator, test_generator, scaler, combined_data, train_loader, test_loader

## Create GRU Model

In [ ]:
def create_gru_model(combined_data, print_summary=True):

  model = Sequential([
      Input(shape=(CONTEXT_WINDOW, combined_data.shape[1])),
      GRU(units=64,
          activation='relu',
          return_sequences=True,
          kernel_regularizer=L2(0.001)
      ),
      GRU(units=32,
          activation='relu',
          recurrent_dropout=0.25
      ),
      Dense(32, activation='relu'),
      Dense(1, activation='linear')
  ])

  OPTIMIZER = Adam(learning_rate=LEARNING_RATE_LSTM_GRU)
  model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=[METRICS])

  if print_summary:
    model.summary()

  return model

## Create LSTM Model

In [ ]:
def create_lstm_model(combined_data, print_summary=True):

  model = Sequential([
    Input(shape=(CONTEXT_WINDOW, combined_data.shape[1])),
    LSTM(units=64,
          activation='relu',
          return_sequences=True,
          kernel_regularizer=L2(0.001),
    ),
    LSTM(units=32,
          activation='relu',
          recurrent_dropout=0.25
    ),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
  ])

  OPTIMIZER = Adam(learning_rate=LEARNING_RATE_LSTM_GRU)
  model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=[METRICS])

  if print_summary:
    model.summary()

  return model

## Create Transformer Model

In [ ]:
def create_transformer_model(combined_data, train_loader):

  class TimeSeriesTransformer(nn.Module):
    def __init__(self, feature_size, num_heads=NUM_TRANSFORMER_HEADS, num_layers=NUM_TRANSFORMER_LAYERS, hidden_dim=NUM_TRANSFORMER_HIDDEN_DIM):
      super().__init__()
      self.input_proj = nn.Linear(feature_size, hidden_dim)
      encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=num_heads, dim_feedforward=2*hidden_dim, batch_first=True)
      self.transformer = nn.TransformerEncoder(encoder_layer=encoder_layer, num_layers=num_layers)
      self.regressor = nn.Linear(hidden_dim, 1)

    def forward(self, x):
      x = self.input_proj(x)
      x = self.transformer(x)
      return self.regressor(x[:, -1, :]).squeeze()

  feature_size = combined_data.shape[1]

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  transformer_model = TimeSeriesTransformer(feature_size=feature_size).to(device)
  criterion = nn.MSELoss()
  optimizer = torch.optim.Adam(transformer_model.parameters(), lr=LEARNING_RATE_TRANSFORMER)

  return transformer_model, criterion, optimizer

## Define Second Layer Model

In [ ]:
def create_second_layer_model(input_data_second_layer):
  second_layer_model = Sequential([
      Input(shape=(input_data_second_layer.shape[1],)),
      Dense(10, activation='relu'),
      Dense(5, activation='relu'),
      Dense(1)
  ])

  OPTIMIZER = Adam(learning_rate=LEARNING_RATE_LSTM_GRU)
  second_layer_model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=[METRICS])

  return second_layer_model

# Load the Data

In [ ]:
final_df = pd.read_csv("final_bmri_dataset.csv")
# final_df = pd.read_csv("final_bbri_dataset.csv")
# final_df = pd.read_csv("final_bbca_dataset.csv")

df_processed = parse_embedding_from_df(final_df, EMBEDDING_COLUMN_NAME, n_comp=N_COMPONENT_PCA)

# Create Data for Training and Testing

In [ ]:
train_generator, test_generator, scaler, combined_data, train_loader, test_loader = create_data(
    df_processed=df_processed,
    use_pca=USE_PCA,
    embedding_column_name=EMBEDDING_COLUMN_NAME,
    relevance_column = RELEVANCE_COLUMN_USED
    )

In [ ]:
# lstm error
sum_mse_lstm = 0
sum_mae_lstm = 0
sum_mape_lstm = 0

# gru error
sum_mse_gru = 0
sum_mae_gru = 0
sum_mape_gru = 0

# transformer error
sum_mse_transformer = 0
sum_mae_transformer = 0
sum_mape_transformer = 0

# second layer eval
sum_mse_second_layer = 0
sum_mae_second_layer = 0
sum_mape_second_layer = 0

for i in tqdm(range(N_ITERATION), desc='training and evaluating model'):

  # create and train the LSTM model
  lstm_model = create_lstm_model(combined_data, print_summary=False)

  early_stopping = EarlyStopping(
      monitor='loss', # Monitor train loss
      patience=10,        # Number of epochs with no improvement after which training will be stopped.
      restore_best_weights=True # Restore model weights from the epoch with the best value of the monitored quantity.
  )

  history = lstm_model.fit(
      train_generator,
      epochs=NUM_EPOCHS,
      verbose=0,
      callbacks=[early_stopping] # Add the early stopping callback here
  )

  # create and train the GRU model
  gru_model = create_gru_model(combined_data, print_summary=False)

  early_stopping = EarlyStopping(
      monitor='loss', # Monitor train loss
      patience=10,        # Number of epochs with no improvement after which training will be stopped.
      restore_best_weights=True # Restore model weights from the epoch with the best value of the monitored quantity.
  )

  history = gru_model.fit(
      train_generator,
      epochs=NUM_EPOCHS,
      verbose=0,
      callbacks=[early_stopping] # Add the early stopping callback here
  )

  transformer_model, criterion, optimizer = create_transformer_model(combined_data, train_loader)

  # Check for GPU availability and set the device
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  # model training
  for epoch in range(NUM_EPOCHS):
    transformer_model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
      X_batch, y_batch = X_batch.to(device), y_batch.to(device)
      optimizer.zero_grad()
      outputs = transformer_model(X_batch)
      loss = criterion(outputs, y_batch)
      loss.backward()
      optimizer.step()
      total_loss += loss.item()

  actual_train_data = np.array(train_generator.targets[CONTEXT_WINDOW:])

  # making prediction for second layer training data
  lstm_train_predictions = lstm_model.predict(train_generator, verbose=0)
  gru_train_predictions = gru_model.predict(train_generator, verbose=0)

  transformer_model.eval()
  transformer_train_predictions = []
  with torch.no_grad():
    for X_batch, y_batch in train_loader:
      X_batch = X_batch.to(device)
      outputs = transformer_model(X_batch)
      transformer_train_predictions.extend(outputs.cpu().numpy())

  transformer_train_predictions = np.array(transformer_train_predictions).reshape(-1, 1)

  # concatenate all the train prediction data to form the complete training data for second layer model
  predicted_train = np.concatenate((lstm_train_predictions, gru_train_predictions, transformer_train_predictions), axis=1)

  # create second layer model
  second_layer_model = create_second_layer_model(predicted_train)

  # train second layer model
  second_layer_model.fit(predicted_train, actual_train_data, epochs=NUM_EPOCHS, verbose=0)

  # making prediction on test data using lstm and gru
  lstm_test_predictions = lstm_model.predict(test_generator, verbose=0)
  gru_test_predictions = gru_model.predict(test_generator, verbose=0)

  # making prediction on test data using transformer
  transformer_model.eval()
  transformer_test_predictions, trues = [], []
  with torch.no_grad():
    for X_batch, y_batch in test_loader:
      X_batch, y_batch = X_batch.to(device), y_batch.to(device) # Move data to device
      outputs = transformer_model(X_batch)
      transformer_test_predictions.extend(outputs.cpu().numpy()) # Move predictions back to CPU for evaluation
      trues.extend(y_batch.cpu().numpy()) # Move true values back to CPU for evaluation

  transformer_test_predictions = np.array(transformer_test_predictions).reshape(-1, 1)

  # concatenate the predictions from three base model to form single dataset
  predicted_test = np.concatenate((lstm_test_predictions, gru_test_predictions, transformer_test_predictions), axis=1)

  lstm_test_predictions_actual = scaler.inverse_transform(lstm_test_predictions)
  gru_test_predictions_actual = scaler.inverse_transform(gru_test_predictions)
  transformer_test_predictions_actual = scaler.inverse_transform(transformer_test_predictions)

  second_layer_test_predictions = second_layer_model.predict(predicted_test, verbose=0)

  predicted_actual = scaler.inverse_transform(second_layer_test_predictions)
  true_actual = scaler.inverse_transform(test_generator.targets.reshape(-1, 1))

  # trim the true actual to be the same length
  true_actual = true_actual[CONTEXT_WINDOW:]

  # lstm eval
  sum_mse_lstm += mean_squared_error(true_actual, lstm_test_predictions_actual)
  sum_mae_lstm += mean_absolute_error(true_actual, lstm_test_predictions_actual)
  sum_mape_lstm += mean_absolute_percentage_error(true_actual, lstm_test_predictions_actual)

  # gru eval
  sum_mse_gru += mean_squared_error(true_actual, gru_test_predictions_actual)
  sum_mae_gru += mean_absolute_error(true_actual, gru_test_predictions_actual)
  sum_mape_gru += mean_absolute_percentage_error(true_actual, gru_test_predictions_actual)

  # transformer eval
  sum_mse_transformer += mean_squared_error(true_actual, transformer_test_predictions_actual)
  sum_mae_transformer += mean_absolute_error(true_actual, transformer_test_predictions_actual)
  sum_mape_transformer += mean_absolute_percentage_error(true_actual, transformer_test_predictions_actual)

  # second layer eval
  sum_mse_second_layer += mean_squared_error(true_actual, predicted_actual)
  sum_mae_second_layer += mean_absolute_error(true_actual, predicted_actual)
  sum_mape_second_layer += mean_absolute_percentage_error(true_actual, predicted_actual)

In [ ]:
average_mse_lstm = sum_mse_lstm / N_ITERATION
average_mae_lstm = sum_mae_lstm / N_ITERATION
average_mape_lstm = sum_mape_lstm / N_ITERATION

average_mse_gru = sum_mse_gru / N_ITERATION
average_mae_gru = sum_mae_gru / N_ITERATION
average_mape_gru = sum_mape_gru / N_ITERATION

average_mse_transformer = sum_mse_transformer / N_ITERATION
average_mae_transformer = sum_mae_transformer / N_ITERATION
average_mape_transformer = sum_mape_transformer / N_ITERATION

average_mse_second_layer = sum_mse_second_layer / N_ITERATION
average_mae_second_layer = sum_mae_second_layer / N_ITERATION
average_mape_second_layer = sum_mape_second_layer / N_ITERATION

print("LSTM TEST RESULT")
print(f"Test MSE: {average_mse_lstm:.3f}")
print(f"Test MAE: {average_mae_lstm:.3f}")
print(f"Test MAPE: {(average_mape_lstm*100):.2f}%")
print("\n")

print("GRU TEST RESULT")
print(f"Test MSE: {average_mse_gru:.3f}")
print(f"Test MAE: {average_mae_gru:.3f}")
print(f"Test MAPE: {(average_mape_gru*100):.2f}%")
print("\n")

print("TRANSFORMER TEST RESULT")
print(f"Test MSE: {average_mse_transformer:.3f}")
print(f"Test MAE: {average_mae_transformer:.3f}")
print(f"Test MAPE: {(average_mape_transformer*100):.2f}%")
print("\n")

print("ENSEMBLE MODEL TEST RESULT")
print(f"Test MSE: {average_mse_second_layer:.3f}")
print(f"Test MAE: {average_mae_second_layer:.3f}")
print(f"Test MAPE: {(average_mape_second_layer*100):.2f}%")

# Visualize the results (predicted vs actual)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(true_actual, label='Actual Price')
plt.plot(predicted_actual, label='Predicted Price (Ensemble Model)')
plt.title('Actual vs. Predicted Stock Price (BMRI)')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.grid(True)
plt.show()